## Extract Binance financial data Cardano
In this notebook we will use the 'python-binance' package to extract financial data from Binance' API. For this to work one should have their own Binance account and request an API key and secret password. We are looking to extract financial data from Cardano with different time frames (1m, 5m, 1h).

In [1]:
#!pip install python-binance

import numpy as np
import pandas as pd
import math
import os.path
import time
from binance.client import Client
from datetime import timedelta, datetime
from dateutil import parser


### API

binance_api_key = '[ENTER YOUR BINANCE API KEY]'
binance_api_secret = '[ENTER YOUR BINANCE API SECRET KEY]'

### CONSTANTS
binsizes = {"1m": 1, "5m": 5, "1h": 60}
batch_size = 750
binance_client = Client(api_key=binance_api_key, api_secret=binance_api_secret)


### FUNCTIONS
def minutes_of_new_data(symbol, kline_size, data, source):
    if len(data) > 0:  old = parser.parse(data["timestamp"].iloc[-1])
    elif source == "binance": old = datetime.strptime('1 Jan 2017', '%d %b %Y')
    if source == "binance": new = pd.to_datetime(binance_client.get_klines(symbol=symbol, interval=kline_size)[-1][0], unit='ms')
    return old, new

def get_all_binance(symbol, kline_size, save = False):
    filename = '%s-%s-data.csv' % (symbol, kline_size)
    if os.path.isfile(filename): data_df = pd.read_csv(filename)
    else: data_df = pd.DataFrame()
    oldest_point, newest_point = minutes_of_new_data(symbol, kline_size, data_df, source = "binance")
    delta_min = (newest_point - oldest_point).total_seconds()/60
    available_data = math.ceil(delta_min/binsizes[kline_size])
    if oldest_point == datetime.strptime('1 Jan 2017', '%d %b %Y'): print('Downloading all available %s data for %s. Be patient..!' % (kline_size, symbol))
    else: print('Downloading %d minutes of new data available for %s, i.e. %d instances of %s data.' % (delta_min, symbol, available_data, kline_size))
    klines = binance_client.get_historical_klines(symbol, kline_size, oldest_point.strftime("%d %b %Y %H:%M:%S"), newest_point.strftime("%d %b %Y %H:%M:%S"))
    data = pd.DataFrame(klines, columns = ['timestamp', 'open', 'high', 'low', 'close', 'volume', 'close_time', 'quote_av', 'trades', 'tb_base_av', 'tb_quote_av', 'ignore' ])
    data['timestamp'] = pd.to_datetime(data['timestamp'], unit='ms')
    if len(data_df) > 0:
        temp_df = pd.DataFrame(data)
        data_df = data_df.append(temp_df)
    else: data_df = data
    data_df.set_index('timestamp', inplace=True)
    if save: data_df.to_csv(filename)
    print('All caught up..!')
    return data_df


In [2]:
ADAUSDT_1h = get_all_binance('ADAUSDT', "1h", save = True )


ADAUSDT_1h

All caught up..!


,open,high,low,close,volume,close_time,quote_av,trades,tb_base_av,tb_quote_av,ignore
timestamp,,,,,,,,,,,
2018-04-17 04:00:00,0.25551,0.288,0.25551,0.26664,8.14369e+06,1523941199999,2.16508e+06,4421,2.88982e+06,767134,0
2018-04-17 05:00:00,0.2666,0.27798,0.2601,0.262,8.31792e+06,1523944799999,2.23563e+06,5149,3.81953e+06,1.02525e+06,0
2018-04-17 06:00:00,0.26221,0.26396,0.248,0.25664,8.4201e+06,1523948399999,2.15396e+06,4566,3.63615e+06,930293,0
2018-04-17 07:00:00,0.25662,0.263,0.25489,0.25698,4.68604e+06,1523951999999,1.21562e+06,3283,1.6332e+06,423445,0
2018-04-17 08:00:00,0.25636,0.25998,0.25229,0.25631,3.51004e+06,1523955599999,896096,2408,1.34922e+06,344486,0
...,...,...,...,...,...,...,...,...,...,...,...
2021-05-16 18:00:00,2.23440000,2.25380000,2.12000000,2.12760000,55472585.71000000,1621191599999,121262892.46818100,159310,26216860.86000000,57374697.43727500,0
2021-05-16 19:00:00,2.12710000,2.16510000,2.02000000,2.09120000,115093668.11000000,1621195199999,241772027.58008000,223820,54761723.02000000,115124761.19119500,0
2021-05-16 20:00:00,2.09170000,2.12320000,2.01040000,2.02620000,88952474.14000000,1621198799999,183739720.18163500,181026,45234577.00000000,93488639.72659000,0
